#import and install

#Task


**Задание: реализовать детектор сигареты на изображениях/видео (на выходе -
bounding box сигареты).**   
1. Дататест  с фото курильщков взят  kaggle
2. Проведена разметка  для детекции  с помощью https://www.makesense.ai/ . около 1200 изображений
3. Выбрана легковесная модель yolov5s .
4. ДОбучена модель на весах COCO датасета на 35 эпохах.(больше уже не хватет бесплатыного времени GPU   в colab)
5. Уже на наших весах делаем предикт тест дата
6. Для улучшения метрик необходимо :
- больше размеченных данных
- железо для более долго обучения, необходимо около 300 эпох миниум
7. В целом получена неплохая детекция с такого короткого обучения.

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-187-g0004c74 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [12]:
#подключить гугл диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
import shutil

In [10]:
PATH = '/content/drive/MyDrive/DS/kaggle/cigarette-detection/'
PATH_yolo = '/content/yolov5/'
PATH_weigth = '/content/drive/MyDrive/DS/kaggle/cigarette-detection/weights/'

#Подготовка файлов

In [ ]:
# train dataset conifg copy to yolo dir
!cp {PATH}cigarette-detection.yaml  {PATH_yolo}data/cigarrete-detection.yaml

In [ ]:
# unzip  train dataset
filename = f'{PATH}cigarette-detection.zip'
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(f'{PATH}')

In [ ]:
ls /content/drive/MyDrive/DS/kaggle/cigarette-detection/

cigarette-detection.yaml  images/       predicted_data_v1/          video/
cigarette-detection.zip   labels/       predicted_test_data_v2.zip  weights/
cig_sm.mp4                labels.cache  test_data/                  yolov5/


#Train    


In [ ]:
!python train.py --img 640 --epochs 35 --data cigarrete-detection.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=cigarrete-detection.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=35, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-185-g2334aa7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7

In [ ]:
# copy weigts
!cp   /content/yolov5/runs/train/exp4/weights/best.pt {PATH_weigth}cigarette_v2.pt

#Validation

In [ ]:
!python val.py --weights {PATH_weigth}best_v2.pt --data cigarrete-detection.yaml --img 640 --half

val: data=/content/yolov5/data/cigarrete-detection.yaml, weights=['/content/drive/MyDrive/DS/kaggle/cigarette-detection/weights/best_v2.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-185-g2334aa7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/drive/MyDrive/DS/kaggle/cigarette-detection/labels.cache... 1173 images, 0 backgrounds, 6 corrupt: 100% 1179/1179 [00:00<?, ?it/s]
train: WARNING ⚠️ /content/drive/MyDrive/DS/kaggle/cigarette-detection/images/000031.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/DS/kaggle/cigarette-detection/images/000083.jpg: corrupt JPEG restored and sav

#Detect on test data

In [ ]:
!python detect.py --weights {PATH_weigth}cigarette_v2.pt --img 640 --conf 0.20 --exist-ok   --source {PATH}test_data

detect: weights=['/content/drive/MyDrive/DS/kaggle/cigarette-detection/weights/cigarette_v2.pt'], source=/content/drive/MyDrive/DS/kaggle/cigarette-detection/test_data, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-186-g0acc5cf Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/187 /content/drive/MyDrive/DS/kaggle/cigarette-detection/test_data/00000.jpg: 416x640 1 cigarrete, 42.2ms
image 2/187 /content/drive/MyDrive/DS/kaggle/cigarette-detection/test_data/00001.jpg: 448x640 1 cigarrete, 80.1ms
image 3/187 /content/drive/MyDr

In [ ]:
!python detect.py --weights {PATH_weigth}best_v2.pt --img 640 --conf 0.45 --name "$(pwd)"   --source {PATH}/video/blonde.mp4

Выходные данные были обрезаны до нескольких последних строк (5000).
video 1/1 (423/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detection/video/blonde.mp4: 384x640 (no detections), 6.5ms
video 1/1 (424/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detection/video/blonde.mp4: 384x640 (no detections), 6.6ms
video 1/1 (425/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detection/video/blonde.mp4: 384x640 (no detections), 8.1ms
video 1/1 (426/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detection/video/blonde.mp4: 384x640 (no detections), 6.5ms
video 1/1 (427/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detection/video/blonde.mp4: 384x640 (no detections), 6.5ms
video 1/1 (428/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detection/video/blonde.mp4: 384x640 (no detections), 6.5ms
video 1/1 (429/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detection/video/blonde.mp4: 384x640 (no detections), 6.5ms
video 1/1 (430/5420) /content/drive/MyDrive/DS/kaggle/cigarette-detecti

#save predicted data

In [ ]:
directory_path = '/content/yolov5/runs/detect/exp'
zip_path = '/content/yolov5/runs/detect/exp'

shutil.make_archive(zip_path, 'zip', directory_path)

'/content/yolov5/runs/detect/exp.zip'

In [ ]:
!cp     /content/yolov5/runs/detect/exp.zip    {PATH}predicted_test_data.zip
!cp     /content/yolov54/blonde.mp4   {PATH}predicted_blonde_v2.mp4


In [ ]:
ls {PATH}

cigarette-detection.yaml  labels.cache                predicted_test_data.zip
cigarette-detection.zip   predicted_blonde.mp4        test_data/
cig_sm.mp4                predicted_blonde_v2.mp4     video/
images/                   predicted_data_v1/          weights/
labels/                   predicted_test_data_v2.zip  yolov5/


### for task

In [5]:
pwd

'/content/yolov5'

In [8]:
!mkdir weights

mkdir: cannot create directory ‘weights’: File exists


In [13]:
!cp {PATH_weigth}cigarette_v2.pt /content/yolov5/weights/cigarette_v2.pt

In [16]:
!python detect.py --weights weights/cigarette_v2.pt --img 640 --conf 0.25 --exist-ok  --name "$(pwd)" --source 000002.jpg

detect: weights=['weights/cigarette_v2.pt'], source=000002.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=/content/yolov5, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-187-g0004c74 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/yolov5/000002.jpg: 384x640 1 cigarrete, 48.7ms
Speed: 0.7ms pre-process, 48.7ms inference, 240.9ms NMS per image at shape (1, 3, 640, 640)
Results saved to /content/yolov5
